## Table of Contents

* [0 Grid information](#0_Grid_information)

In [ ]:
######## LOBAFLEX ###########
# from lobaflex.analysis import analysis
# from lobaflex.tools.draw import *
from lobaflex.opt.feeder_extraction import get_flexible_loads
from lobaflex.tools.tools import get_config, get_files_in_subdirs
from lobaflex.opt.dispatch_integration import integrate_dispatch
from edisgo.edisgo import import_edisgo_from_files

######## DIRS ##########
from lobaflex import config_dir, data_dir, results_dir
######## DATA ###########
import re
import pandas as pd

######## PLOTTING ###########
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# # set seaborn style
# sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

# comment to make plots interactive
pio.renderers.default = "svg"



In [ ]:
cfg_o = get_config(path=config_dir / ".opt.yaml")

## 0 Grid information<a class="anchor" id="0_Grid_information"></a>

# Definitions

In [ ]:
# grid_path = data_dir / import_dir/ str(grid_id)
grid_path = results_dir / run_id / str(grid_id) / "minimize_loading" / "reinforced"

results_path = results_dir / run_id / str(grid_id) / "minimize_loading" / "concat"

list_of_results = get_files_in_subdirs(path=results_path, pattern="*.csv")

# Import edisgo

In [ ]:
edisgo_obj = import_edisgo_from_files(
    edisgo_path=grid_path,
    import_topology=True,
    import_timeseries=True,
    import_electromobility=True,
    import_heat_pump=True,
)

In [ ]:
if start_datetime and periods is not None:
    timeframe = pd.date_range(start=start_datetime, periods=periods, freq="1h")
else:
    timeframe = edisgo_obj.timeseries.timeindex

# Generator and load ids

In [ ]:
flexible_loads = edisgo_obj.topology.loads_df.loc[edisgo_obj.topology.loads_df["opt"] == True].index
inflexible_loads = edisgo_obj.topology.loads_df.loc[~edisgo_obj.topology.loads_df.index.isin(flexible_loads)].index

In [ ]:
res_generators = edisgo_obj.topology.generators_df[edisgo_obj.topology.generators_df["type"].isin(
    ["solar", "wind"])].index

In [ ]:
flexible_hp = [i for i in flexible_loads if "HP" in i]
flexible_cp = [i for i in flexible_loads if "Charging_Point" in i]

# Plots

## curtailment

In [ ]:
curtailment_files = [i for i in list_of_results if "curtailment" in i]

In [ ]:
n = len(curtailment_files)
fig, axs = plt.subplots(nrows=n, figsize=(15,n*5), sharex=True)
for i, file in enumerate(curtailment_files):
    attr = re.findall(r'curtailment_(.*).csv', file)[0]
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    df = df.sum(axis=1).loc[timeframe].rename(f"Curtailment {attr}")
    sns.lineplot(x=timeframe, y=df, ax=axs[i])
# fig.show()

## slacks

## Charging + feedin

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    mode= "lines",
    name = 'Heat pumps',
    x = timeframe,
    y = edisgo_obj.timeseries.loads_active_power.loc[timeframe, flexible_hp].sum(axis=1),
#     stackgroup='one'
    yaxis='y1',
))

fig.add_trace(go.Scatter(
    mode= "lines",
    name = 'Charging points',
    x = timeframe,
    y = edisgo_obj.timeseries.loads_active_power.loc[timeframe, flexible_cp].sum(axis=1),
#     stackgroup='one',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    mode= "lines",
    name = 'Feedin-fix_load',
    x = timeframe,
    y = edisgo_obj.timeseries.generators_active_power.loc[timeframe, res_generators].sum(axis=1) - \
    edisgo_obj.timeseries.loads_active_power.loc[timeframe, inflexible_loads].sum(axis=1),
#     stackgroup='two',
    yaxis='y2',
))


fig.update_layout(
    title=f"Charging: {run_id} - {grid_id}",
    width=1000,
    height=600,
    yaxis_title="MW",
    xaxis_title="timesteps",
    showlegend=True,
    yaxis=dict(title='Power in MW'),
    yaxis2=dict(title='Power in MW',
                overlaying='y',side='right')
)

fig.show()



# Plot dumb + opt charging

# Plot energy level + charging